# Простые счётчики на службе векторной семантики
## Скачиваем текст

In [2]:
import urllib
from urllib import request
from tqdm import tqdm



with tqdm(...) as t:
    text_data = request.urlopen("http://www.gutenberg.org/files/1399/1399-0.txt").read().decode("UTF-8") + " "
    
# with tqdm(...) as t:
#     text_data += request.urlopen("https://archive.org/stream/warandpeace030164mbp/warandpeace030164mbp_djvu.txt").read().decode("UTF-8")

#text_data[:100]

ModuleNotFoundError: No module named 'progressbar'

## Убираем теги

In [2]:
with open('warandpeace.txt') as f:
    text_data = f.read()

In [3]:
import re

clean_data = re.sub("<[^>]*>", " ", text_data)
clean_data = re.sub("\s+", " ", text_data)
clean_data[:100]

'Skip to main content web texts movies audio software image logosearch Search upload personSIGN IN AB'

In [ ]:
import nltk
nltk.download('punkt')

## Бьём текст на предложения

In [6]:
import nltk
import nltk.data

# загружаем токенизатор
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = sent_detector.tokenize(clean_data.strip())

print("Total number of sentences:", len(sentences))

sentences[:4]

Total number of sentences: 36424


['Skip to main content web texts movies audio software image logosearch Search upload personSIGN IN ABOUT CONTACT BLOG PROJECTS HELP DONATE JOBS VOLUNTEER PEOPLE Full text of "War And Peace" See other formats TEXT FLY WITHIN THE BOOK ONLY Tight Binding Book CO > UJ ft <OU_1 68052 >m OUP 2273 19-1 1-79 10,000 Copies.',
 "OSMANIA UNIVERSITY LIBRARY Call No ^^L Accession No ^95 gg Author -j- 4.STt> y^ Title W<U *'<P4>*' ( *~' This book should bc^rcturned on or before the date last marked below.",
 'War and Peace BY LEO TOLSTOY Translated b\\ LOUISE and AYLMER MAUDE WILLIAM BENTON, Publisher ENCYCLOPEDIA BR1TANNICA, INC. CHICAGO - LONDON - TORONTO BY ARRANGEMENT WITH OXFORD UNIVERSITY PRESS COPYRIGHT IN THE UNITED STATES OF AMERICA, 1952, BY ENCYCLOPEDIA BRITANNICA,INC.',
 'COPYRIGHT 1952.']

In [10]:
sentences[1000:1040]

['The elderly lady was a Princess Drubet- skdya, belonging to one of the best families in Russia, but she was poor, and having long been out of society had lost her former influential connections.',
 'She had now come to Petersburg to procure an appointment in the Guards for her only son.',
 "It was, in fact, solely to meet Prince Vasfli that she had obtained an invita- tion to Anna Pdvlovna's reception and had sat listening to the vicomte's story.",
 "Prince Vasfli's words frightened her, an embittered look clouded her once handsome face, but only for a moment; then she smiled again and clutched Prince Vasili's arm more tightly.",
 '"Listen to me, Prince," said she.',
 '"I have never yet asked you for anything and I never will again, nor have I ever reminded you of my father\'s friendship for you; but now I en- treat you for God\'s sake to do this for my son and I shall always regard you as a benefac- tor," she added hurriedly.',
 '"No, don\'t be angry, but promise!',
 'I have asked G

## Разбиваем предложения на токены

In [11]:
%%time
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import re

# nltk.download('wordnet')

# можно попробовать и какой-нибудь другой
lemmatizer = WordNetLemmatizer()

# можно попробовать и какой-нибудь другой
tokenizer = TweetTokenizer()
splitted = []

for sent in sentences:
    splitted.append([lemmatizer.lemmatize(w).lower() for w in tokenizer.tokenize(sent) if re.match("^[A-Za-z'-]+$", w)])


## Своеобразный способ построить словарь

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '), min_df=25)
term_doc_matrix = vectorizer.fit_transform([" ".join(sentence) for sentence in splitted])

term2id = vectorizer.vocabulary_
id2term = {v: k for k, v in term2id.items()}

print("Term-document matrix shape:", term_doc_matrix.shape)
print("Vocabulary samples:", list(term2id.items())[:5])

Term-document matrix shape: (36424, 2077)
Vocabulary samples: [('to', 1882), ('main', 1103), ('in', 943), ('about', 7), ('help', 861)]


In [13]:
type(term2id)
term_doc_matrix.shape

(36424, 2077)

## Посчитаем частоты термов, используя построенную матрицу

In [14]:
term_counts = term_doc_matrix.sum(axis=0).A1
print(term_counts.shape)

(2077,)


---- 

### Опциональное задание: выбросьте слишком частотные и слишком редкие термы

----

In [ ]:
#todo: hint: надо использовать term_counts, term2id + перестроить все матрицы
import matplotlib.pyplot as plt
term_counts
# plt.hist(term2id.items())

## Строим term-context matrix

In [ ]:
import numpy as np
import scipy.sparse as sp

half_window = 5

X = np.zeros(shape=(len(term2id), len(term2id)))


for sentence in splitted:
    
    # бежим с индексом по предложению
    for i in range(len(sentence)):        
        current_word = sentence[i]      
        
        # если слова нет в словаре, ничего для него не считаем
        if current_word in term2id:
            word_idx = term2id[current_word]

            for c in range(-half_window, half_window):
                current_idx = i + c

                # проверяем, не наткнулись ли на границы предложения
                if 0 <= current_idx < len(sentence):
                    context_word = sentence[current_idx]
                    
                    if context_word in term2id:                    
                        context_idx = term2id[context_word]
                        X[word_idx, context_idx] += 1

print("Sparsity of the term-context matrix", len(X.nonzero()[0]) / (X.shape[0] ** 2))

## Функция поиска ближайших K

In [ ]:
from scipy.spatial.distance import cdist

def dict_k_closest(M, term_dict, inverse_term_dict, k=5):
    """
        :param M -- матрица векторых представлений
        :param term_dict -- слово2индекс
        :param inverse_term_dict -- индекс2слово
        :param k -- число ближайших соседей для выдачи
    """
    
    print("Computing all distances... (takes some time)")    
    distances = cdist(M, M, "cosine")
    sorted_by_dist_k = np.argsort(distances, axis=1)[:, :k]
    
    results = {}
    
    for term in term_dict:
        row_id = term_dict[term]
        similar = [inverse_term_dict[i] for i in sorted_by_dist_k[row_id, :]]
        results[term] = similar   
        
    return results

## Смотрим глазами на наши успехи

In [ ]:
similar_terms = dict_k_closest(X, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

## Ну как вам?

(Правильный ответ: не очень)


### PMI: Pointwise Mutual Information
    

In [ ]:
X_PMI = X.copy()
total_bicount = X_PMI.sum()
total_unicount = term_counts.sum()

# p(x,y) / p(x) / p(y)
X_PMI = (X_PMI / total_bicount) / (term_counts[:, None] + 0.000001) / (term_counts[None, :] + 0.000001)  * total_unicount ** 2
X_PMI = np.where(X_PMI > 0, np.log2(X_PMI), 0)

In [ ]:
similar_terms = dict_k_closest(X_PMI, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

## Positive PMI

In [ ]:
X_PPMI = X_PMI.copy()
X_PPMI[X_PPMI < 0] = 0.0

In [ ]:
similar_terms = dict_k_closest(X_PPMI, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

### Ну как?
Должно стать лучше

## Применим Truncated SVD

In [ ]:
# U, S, Vh = np.linalg.svd(X_PPMI, full_matrices=False)
# U.shape, S.shape, Vh.shape

In [ ]:
from  sklearn.decomposition import TruncatedSVD

factorizer = TruncatedSVD(n_components=200, random_state=0)

# матрица слов
W = factorizer.fit_transform(X_PPMI)
print(W.shape)

# матрица контекстов
C = factorizer.components_
print(C.shape)

In [ ]:
similar_terms = dict_k_closest(W, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

### Попробуем разложить в произведение двух матриц

In [ ]:
from  sklearn.decomposition import NMF

factorizer = NMF(n_components=200, random_state=0)

# матрица слов
W = factorizer.fit_transform(X_PPMI)
print(W.shape)

# матрица контекстов
C = factorizer.components_
print(C.shape)

In [ ]:
similar_terms = dict_k_closest(W, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

##  Задача техническая: сохранить векторы в CSV
---- 

# Посмотрим, как можно оценивать качество
Отличный источник, горячо рекомендуется
https://github.com/EloiZ/embedding_evaluation

Надо склонировать репозиторий, загрузить датасеты с помощью
`download_benchmarks.py`

In [ ]:
import os

os.environ["EMBEDDING_EVALUATION_DATA_PATH"] = "embedding_evaluation/data/"

import embedding_evaluation
from embedding_evaluation.evaluate import Evaluation
from embedding_evaluation.load_embedding import load_embedding_textfile

def eval_word_vectors(path):
    # Load embeddings as a dictionnary {word: embed} where embed is a 1-d numpy array.
    embeddings = load_embedding_textfile(textfile_path=path)

    # Load and process evaluation benchmarks
    evaluation = Evaluation() 

    return evaluation.evaluate(embeddings)

## Задача: оценить качество сохранённых моделей, попытаться его улучшить

In [ ]:
mymodelresults = eval_word_vectors("trali-vali.csv")

mymodelresults

In [9]:
splitted[0]

['skip',
 'to',
 'main',
 'content',
 'web',
 'text',
 'movie',
 'audio',
 'software',
 'image',
 'logosearch',
 'search',
 'upload',
 'personsign',
 'in',
 'about',
 'contact',
 'blog',
 'projects',
 'help',
 'donate',
 'jobs',
 'volunteer',
 'people',
 'full',
 'text',
 'of',
 'war',
 'and',
 'peace',
 'see',
 'other',
 'format',
 'text',
 'fly',
 'within',
 'the',
 'book',
 'only',
 'tight',
 'binding',
 'book',
 'co',
 'uj',
 'ft',
 'm',
 'oup',
 'copies']